# Momentum Strategy

In [1]:
# import yfinance as yf
import pandas as pd
import numpy as np
import yfinance as yf

## Getting the data

In [2]:
spy = pd.read_excel('./data/spy_components.xlsx')
ticker = spy['Ticker'].dropna().tolist()
ticker = ' '.join(ticker)
ticker = ticker.replace('.', '-')
data = yf.download(ticker, start="2014-01-01")
# data.to_csv('./data/spy_data.csv')
close = data['Adj Close']


[*********************100%***********************]  504 of 504 completed

1 Failed download:
- CASH_USD: No data found for this date range, symbol may be delisted


In [14]:
import plotly.graph_objects as go
from plotly.offline import init_notebook_mode, iplot

init_notebook_mode(connected=True)  
 
# create figure
fig = go.Figure()
 
# add trace
fig.add_trace(
    go.Scatter(
        x = close.index,
        y = close["MSFT"],
        line = dict(color="blue", width=2),
        name = "Microsoft"
        )
    )
 
# update layout
fig.update_layout(
    title = "Stockprice Microsoft"
    )
 
# show figure
iplot(fig)   

## Resample Data

In [4]:
def resample_prices(close_prices, freq='M'):
    """
    Resample close prices for each ticker at specified frequency.
    
    Parameters
    ----------
    close_prices : DataFrame
        Close prices for each ticker and date
    freq : str
        What frequency to sample at
        For valid freq choices, see http://pandas.pydata.org/pandas-docs/stable/timeseries.html#offset-aliases
    
    Returns
    -------
    prices_resampled : DataFrame
        Resampled prices for each ticker and date
    """
    
    return close_prices.resample(freq).last()

monthly_close = resample_prices(close)

## Compute log-returns

In [5]:
def compute_log_returns(prices):
    """
    Compute log returns for each ticker.
    
    Parameters
    ----------
    prices : DataFrame
        Prices for each ticker and date
    
    Returns
    -------
    log_returns : DataFrame
        Log returns for each ticker and date
    """
    
    return np.log(prices/prices.shift(1))

monthly_close_returns = compute_log_returns(monthly_close)

In [16]:
# create figure
fig = go.Figure()
 
# add trace
fig.add_trace(
    go.Scatter(
        x = monthly_close_returns.index,
        y = monthly_close_returns["MSFT"],
        line = dict(color="blue", width=2),
        name = "Microsoft"
        )
    )
 
# update layout
fig.update_layout(
    title = "log-returns Microsoft"
    )
 
# show figure
iplot(fig)   

## Generating the Trading Signal

In [7]:
def shift_returns(returns, shift_n):
    """
    Generate shifted returns
    
    Parameters
    ----------
    returns : DataFrame
        Returns for each ticker and date
    shift_n : int
        Number of periods to move, can be positive or negative
    
    Returns
    -------
    shifted_returns : DataFrame
        Shifted returns for each ticker and date
    """
    
    return returns.shift(shift_n)

prev_returns = shift_returns(monthly_close_returns, 1)
lookahead_returns = shift_returns(monthly_close_returns, -1)

In [8]:
def get_top_n(prev_returns, top_n):
    """
    Select the top performing stocks
    
    Parameters
    ----------
    prev_returns : DataFrame
        Previous shifted returns for each ticker and date
    top_n : int
        The number of top performing stocks to get
    
    Returns
    -------
    top_stocks : DataFrame
        Top stocks for each ticker and date marked with a 1
    """
    top_stocks = prev_returns.applymap(lambda x: 0)

    for date, row in prev_returns.iterrows():
        top_stocks.loc[date, prev_returns.loc[date].nlargest(top_n).index] += 1

    return top_stocks

top_bottom_n = 50
df_long = get_top_n(prev_returns, top_bottom_n)
df_short = get_top_n(-1*prev_returns, top_bottom_n)

## Portfolio Returns

In [9]:
def portfolio_returns(df_long, df_short, lookahead_returns, n_stocks):
    """
    Compute expected returns for the portfolio, assuming equal investment in each long/short stock.
    
    Parameters
    ----------
    df_long : DataFrame
        Top stocks for each ticker and date marked with a 1
    df_short : DataFrame
        Bottom stocks for each ticker and date marked with a 1
    lookahead_returns : DataFrame
        Lookahead returns for each ticker and date
    n_stocks: int
        The number number of stocks chosen for each month
    
    Returns
    -------
    portfolio_returns : DataFrame
        Expected portfolio returns for each ticker and date
    """
    portfolio_returns = df_long*lookahead_returns/n_stocks - \
                            df_short*lookahead_returns/n_stocks
   
    return portfolio_returns

expected_portfolio_returns = portfolio_returns(df_long, df_short, lookahead_returns, 2*top_bottom_n)
portfolio_return = expected_portfolio_returns.sum(axis=1)

In [10]:
# create figure
fig = go.Figure()
 
# add trace
fig.add_trace(
    go.Scatter(
        x = portfolio_return.index,
        y = portfolio_return,
        line = dict(color="blue", width=2),
        name = "returns"
        )
    )
 
# update layout
fig.update_layout(
    title = "log-returns strategy"
    )
 
# show figure
iplot(fig)   

In [11]:
portfolio_ret_mean = portfolio_return.mean()
portfolio_ret_ste = portfolio_return.sem()
portfolio_ret_annual_rate = (np.exp(portfolio_ret_mean * 12) - 1) * 100

print("""
Mean:                       {:.6f}
Standard Error:             {:.6f}
Annualized Rate of Return:  {:.2f}%
""".format(portfolio_ret_mean, portfolio_ret_ste, portfolio_ret_annual_rate))


Mean:                       0.001651
Standard Error:             0.001748
Annualized Rate of Return:  2.00%



## Statistical Significance

In [12]:
from scipy import stats

def analyze_alpha(expected_portfolio_returns_by_date):
    """
    Perform a t-test with the null hypothesis being that the expected mean return is zero.
    
    Parameters
    ----------
    expected_portfolio_returns_by_date : Pandas Series
        Expected portfolio returns for each date
    
    Returns
    -------
    t_value
        T-statistic from t-test
    p_value
        Corresponding p-value
    """
    t_value, p_value = stats.ttest_1samp(expected_portfolio_returns_by_date, 0)
    
    return t_value, p_value/2

t_value, p_value = analyze_alpha(portfolio_return)
print("""
Alpha analysis:
 t-value:        {:.3f}
 p-value:        {:.6f}
""".format(t_value, p_value))


Alpha analysis:
 t-value:        0.944
 p-value:        0.173536

